<a href="https://www.kaggle.com/code/aleksbaran/genai-capstone-project?scriptVersionId=234463133" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Capstone Project

## Smart Job Application Coach (ResumeWhiz)

### Overview

ResumeWhiz is an AI-powered assistant that helps job seekers who struggle with tailoring resumes and writing personalized cover letters for each job application.

This tool uses generative AI to create customized application materials based on a resume and a job description.

### What It Does

- reads and understands job descriptions and resumes
- identifies important skills and requirements
- generates a personalized resume summary
- writes a custom cover letter
- outputs a structured JSON with:
  - match score
  - key strengths
  - gaps
  - suggestions for improvement

### GenAI Capabilities Used

- document understanding
- few-shot prompting
- structured output (JSON)


## Setting up: Import and Gemini configuration

In [1]:
# Clean and install the latest GenAI SDK
!pip uninstall -qqy jupyterlab
!pip install -U -q google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 3.4 MB/s eta 0:00:00


In [2]:
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

# Load your API key securely from Kaggle secrets
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")

# Configure the SDK
genai.configure(api_key=GOOGLE_API_KEY)


In [3]:
# List available models for your account/key
models = genai.list_models()
for m in models:
    print(f"Model: {m.name} — Supported: {m.supported_generation_methods}")


Model: models/chat-bison-001 — Supported: ['generateMessage', 'countMessageTokens']
Model: models/text-bison-001 — Supported: ['generateText', 'countTextTokens', 'createTunedTextModel']
Model: models/embedding-gecko-001 — Supported: ['embedText', 'countTextTokens']
Model: models/gemini-1.0-pro-vision-latest — Supported: ['generateContent', 'countTokens']
Model: models/gemini-pro-vision — Supported: ['generateContent', 'countTokens']
Model: models/gemini-1.5-pro-latest — Supported: ['generateContent', 'countTokens']
Model: models/gemini-1.5-pro-001 — Supported: ['generateContent', 'countTokens', 'createCachedContent']
Model: models/gemini-1.5-pro-002 — Supported: ['generateContent', 'countTokens', 'createCachedContent']
Model: models/gemini-1.5-pro — Supported: ['generateContent', 'countTokens']
Model: models/gemini-1.5-flash-latest — Supported: ['generateContent', 'countTokens']
Model: models/gemini-1.5-flash-001 — Supported: ['generateContent', 'countTokens', 'createCachedContent']
Mo

## Testing Gemini if it works

In [4]:
model = genai.GenerativeModel("gemini-2.0-flash")

response = model.generate_content("Hello! Tell me what is CV.")
print(response.text)


CV stands for **Curriculum Vitae**.

It's a comprehensive document that provides a detailed overview of your professional and academic history. Think of it as a more in-depth resume.

Here's a breakdown of what that generally means:

*   **Detailed and Comprehensive:** A CV includes extensive information about your education, work experience, research, publications, presentations, awards, skills, and other accomplishments.

*   **Academic and Research Focus:** CVs are commonly used in academia, research, and scientific fields. They are also often requested when applying for positions internationally.

*   **Longer Length:** CVs are typically longer than resumes and can range from two pages to multiple pages, depending on the individual's experience and achievements.

**Key Differences between a CV and a Resume:**

| Feature          | Resume                               | CV                                        |
|-----------------|---------------------------------------|-----------

## INPUTS - Resume + Job description

In [5]:
resume_text = """
Experienced software engineer with 5+ years in backend development using Python, Django, and PostgreSQL.
Led a team to build scalable microservices for e-commerce platforms.
Strong experience with REST APIs, CI/CD, and cloud deployment on GCP.
"""

job_description = """
Looking for a backend engineer with experience in Python and Django.
Must have worked with cloud services, especially GCP, and understand CI/CD pipelines.
Experience with microservices architecture is a big plus.
"""

prompt = f"""
You are an expert career coach and recruiter.

Compare the following resume and job description. Provide:
1. A match score from 0-100.
2. Key strengths.
3. Key gaps.
4. Suggestions to improve the resume for this job.

Resume:
{resume_text}

Job Description:
{job_description}
"""

response = model.generate_content(prompt)
print(response.text)

Okay, here's my analysis of the resume against the job description:

**1. Match Score: 85/100**

**2. Key Strengths:**

*   **Python & Django Expertise:** The resume explicitly mentions Python and Django, which are core requirements in the job description. This is a significant point in the resume's favor.
*   **GCP and CI/CD Experience:** Both the resume and the job description emphasize the importance of GCP and CI/CD, indicating a strong alignment.
*   **Microservices Experience:** The resume highlights experience building microservices, which the job description explicitly mentions as a "big plus."
*   **Relevant Experience Level:** The candidate has 5+ years of relevant experience which likely aligns well with the target experience level.
*   **Team Leadership:** The resume mentions leading a team, which suggests leadership potential and the ability to handle complex projects.

**3. Key Gaps:**

*   **Quantifiable Achievements:** While the resume mentions experience, it lacks quan

### Wrapping this up in function

In [6]:
def evaluate_resume_match(resume_text: str, job_description: str, model) -> str:
    prompt = f"""
You are an expert recruiter and career coach.

Compare the following resume and job description. Provide:
1. A match score from 0-100.
2. Key strengths in the resume.
3. Key gaps in the resume for this job.
4. Suggestions to improve the resume to better fit the job.

Resume:
{resume_text}

Job Description:
{job_description}
"""
    response = model.generate_content(prompt)
    return response.text

## Let's test it:

In [7]:
model = genai.GenerativeModel("gemini-2.0-flash")

resume = """
Experienced software engineer with 5+ years in backend development using Python, Django, and PostgreSQL.
Led a team to build scalable microservices for e-commerce platforms.
Strong experience with REST APIs, CI/CD, and cloud deployment on GCP.
"""

job_desc = """
Looking for a backend engineer with experience in Python and Django.
Must have worked with cloud services, especially GCP, and understand CI/CD pipelines.
Experience with microservices architecture is a big plus.
"""

analysis = evaluate_resume_match(resume, job_desc, model)
print(analysis)

Okay, I can analyze this resume and job description and provide a comprehensive assessment.

**1. Match Score:**

I'd give this a **90/100**.

**2. Key Strengths in the Resume:**

*   **Directly Aligns with Core Requirements:** The resume explicitly states proficiency in Python, Django, GCP, and CI/CD pipelines, which are the core requirements of the job.
*   **Microservices Experience:**  Mentions experience with microservices architecture, a "big plus" in the job description.
*   **Leadership Experience:** Highlights leading a team, implying strong collaboration and project management skills.
*   **E-commerce Specific Experience:** Specifies building microservices for e-commerce platforms which can be a differentiating factor.

**3. Key Gaps in the Resume for This Job:**

*   **Quantifiable Achievements:** The resume lacks specific metrics to quantify the impact of the candidate's work. For example, how much did the scalable microservices improve performance?
*   **Specific Tools/Tec

### Let's modify function so it will return JSON format

More compact answer -  2-5 strenght, skilss, gaps and improvement suggestion


In [8]:
import json
import re

def evaluate_resume_match_json(resume_text, job_description_text, model):
    prompt = f"""
You are an expert recruiter and career coach.

Compare the following resume and job description. Return a response strictly in JSON format with the following fields:
{{
  "match_score": int (0-100),
  "strengths": [list of 2-5 key strengths in resume],
  "gaps": [list of 2-5 skill or experience gaps],
  "improvement_suggestions": [list of 2-5 suggestions to improve the resume]
}}

Ensure the output is a well-formed JSON.

Resume:
{resume_text}

Job Description:
{job_description_text}
"""
    response = model.generate_content(prompt)
    
    cleaned_response = str(response.text).strip()
    cleaned_response = re.sub(r'[^\x00-\x7F]+', '', cleaned_response)
    cleaned_response = cleaned_response.strip()
    cleaned_response = cleaned_response.lstrip('\ufeff')

    #print("Cleaned raw response from Gemini:")
    #print(cleaned_response)

    if not cleaned_response:
        print("The model returned an empty response.")
        return None

    cleaned_response = re.sub(r"^```json\s*", "", cleaned_response)
    cleaned_response = re.sub(r"\s*```$", "", cleaned_response)
    
    try:
        json_data = json.loads(cleaned_response)
        print("Parsed JSON:", json_data)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        print("Here's the raw response:")
        print(cleaned_response)
    
    return json_data

In [9]:
result_json = evaluate_resume_match_json(resume, job_desc, model)
print(json.dumps(result_json, indent=2))

Parsed JSON: {'match_score': 85, 'strengths': ['Python and Django proficiency', 'GCP cloud deployment experience', 'CI/CD pipeline experience', 'Microservices architecture experience'], 'gaps': ['Lack of specific quantifiable achievements', 'No mention of testing methodologies'], 'improvement_suggestions': ["Quantify achievements with metrics (e.g., 'Improved API response time by 20%')", 'Specify types of CI/CD tools used (e.g., Jenkins, GitLab CI)', 'Mention testing methodologies (e.g., unit, integration, end-to-end tests)', 'Provide more details on the e-commerce platform and team leadership role']}
{
  "match_score": 85,
  "strengths": [
    "Python and Django proficiency",
    "GCP cloud deployment experience",
    "CI/CD pipeline experience",
    "Microservices architecture experience"
  ],
  "gaps": [
    "Lack of specific quantifiable achievements",
    "No mention of testing methodologies"
  ],
  "improvement_suggestions": [
    "Quantify achievements with metrics (e.g., 'Impro

## File UPLOAD - txt or pdf

In [10]:
!pip install -q pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 76.0 MB/s eta 0:00:00


In [11]:
!pip install -q fpdf

from fpdf import FPDF

# Sample job description text
description = '''
Job Title: Senior Python Developer

Responsibilities:
- Design, develop, and maintain scalable backend systems using Python and Django
- Deploy and manage applications on Google Cloud Platform (GCP)
- Implement CI/CD pipelines and automate deployment processes
- Collaborate with cross-functional teams to define, design, and ship new features
- Ensure code quality through testing and code reviews

Requirements:
- 5+ years of experience with Python and Django
- Experience with GCP cloud services
- Familiarity with microservices architecture
- Strong understanding of CI/CD concepts
- Excellent problem-solving skills
'''

pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 16)
pdf.cell(0, 10, 'Job Description', ln=True, align='C')
pdf.set_font('Arial', '', 12)
for line in description.split('\n'):
    pdf.cell(0, 10, line, ln=True)

pdf.output('job_description.pdf')
'job_description.pdf'

  Preparing metadata (setup.py) ... done


'job_description.pdf'

In [12]:
# Sample CV text
cv_text = '''
Name: Alex Johnson
Email: alex.johnson@email.com
Phone: +1-555-123-4567

Summary:
Experienced Python developer with expertise in Django, GCP, and microservices. Proven track record in building scalable backend systems and automating deployment pipelines.

Experience:
- Backend Developer, E-Shop Inc. (2021-2024)
  * Developed and maintained e-commerce platform using Python/Django
  * Deployed services on GCP, managed CI/CD pipelines
  * Collaborated with frontend and QA teams

- Software Engineer, CloudTech (2018-2021)
  * Built microservices for SaaS products
  * Automated testing and deployment processes

Education:
B.Sc. in Computer Science, Tech University, 2018

Skills:
Python, Django, GCP, CI/CD, Microservices, REST APIs
'''

pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 16)
pdf.cell(0, 10, 'Curriculum Vitae', ln=True, align='C')
pdf.set_font('Arial', '', 12)
for line in cv_text.split('\n'):
    pdf.cell(0, 10, line, ln=True)

pdf.output('cv.pdf')
'cv.pdf'

'cv.pdf'

In [13]:
import pdfplumber

def extract_text_from_file(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = "\n".join([page.extract_text() or "" for page in pdf.pages])
    return text.strip()

resume_path = '/kaggle/working/cv.pdf'
job_desc_path = '/kaggle/working/job_description.pdf'

resume_text = extract_text_from_file(resume_path)
job_desc_text = extract_text_from_file(job_desc_path)

print("Resume extracted text:")
print(resume_text)
print("\nJob Description extracted text:")
print(job_desc_text)

Resume extracted text:
Curriculum Vitae
Name: Alex Johnson
Email: alex.johnson@email.com
Phone: +1-555-123-4567
Summary:
Experienced Python developer with expertise in Django, GCP, and microservices. Proven track record in building scalable backend systems and automating deployment pipelines.
Experience:
- Backend Developer, E-Shop Inc. (2021-2024)
* Developed and maintained e-commerce platform using Python/Django
* Deployed services on GCP, managed CI/CD pipelines
* Collaborated with frontend and QA teams
- Software Engineer, CloudTech (2018-2021)
* Built microservices for SaaS products
* Automated testing and deployment processes
Education:
B.Sc. in Computer Science, Tech University, 2018
Skills:
Python, Django, GCP, CI/CD, Microservices, REST APIs

Job Description extracted text:
Job Description
Job Title: Senior Python Developer
Responsibilities:
- Design, develop, and maintain scalable backend systems using Python and Django
- Deploy and manage applications on Google Cloud Platfor

## Evaluate Resume and Job Description Using Gemini

In [14]:
if resume_text and job_desc_text:
    result_json = evaluate_resume_match_json(resume_text, job_desc_text, model)
else:
    print("Error: Missing resume or job description text.")


Parsed JSON: {'match_score': 85, 'strengths': ['Proficient in Python and Django, aligning with core requirements.', 'Experience with GCP cloud services, matching a key responsibility.', 'Demonstrated ability to implement CI/CD pipelines and automate deployment.', 'Experience developing and maintaining backend systems.'], 'gaps': ["Resume lacks explicit mention of 'problem-solving skills', which the job description requires.", 'Resume does not specify years of experience, making it difficult to assess if the 5+ year requirement is met.', "The resume lacks specific details and metrics around the impact of the candidate's work."], 'improvement_suggestions': ["Quantify achievements in previous roles (e.g., 'Reduced deployment time by 30%').", 'Explicitly state the number of years of experience with Python/Django.', "Add a section or integrate within experience bullet points showcasing problem-solving skills (e.g., 'Debugged and resolved critical performance bottlenecks in the e-commerce pl

In [15]:
result_json = evaluate_resume_match_json(resume_text, job_desc_text, model)

file_path = 'result.json'

try:
    with open(file_path, 'w') as json_file:
        json.dump(result_json, json_file, indent=2)
    print(f"JSON data has been saved to {file_path}")
except Exception as e:
    print(f"An error occurred while saving the file: {e}")

Parsed JSON: {'match_score': 85, 'strengths': ['Strong Python and Django experience', 'Hands-on experience with GCP deployment and management', 'Experience with CI/CD pipeline implementation', 'Experience developing microservices', 'Experience in backend development'], 'gaps': ['Resume does not explicitly state 5+ years of Python/Django experience, although the dates imply it.', 'Lack of explicit mention of testing and code review participation.', 'Resume lacks quantification of accomplishments (e.g., scale of applications, impact of automation).'], 'improvement_suggestions': ['Explicitly state the number of years of Python/Django experience in the summary or experience sections.', 'Add a bullet point detailing participation in code reviews and testing practices.', 'Quantify achievements whenever possible, using metrics to showcase impact.', 'Provide more details on the specific GCP services used and the complexity of the microservices architecture.', "Consider adding a 'Projects' sect

In [16]:
def display_evaluation_result(result_json):
    print("\n" + "="*50)
    print("Resume Evaluation Summary")
    print("="*50)
    print(f"Match Score: {result_json['match_score']} / 100\n")

    print("Strengths:")
    for strength in result_json["strengths"]:
        print(f"  • {strength}")
    
    print("\nGaps:")
    for gap in result_json["gaps"]:
        print(f"  • {gap}")
    
    print("\nSuggestions for Improvement:")
    for suggestion in result_json["improvement_suggestions"]:
        print(f"  • {suggestion}")

In [17]:
display_evaluation_result(result_json)


Resume Evaluation Summary
Match Score: 85 / 100

Strengths:
  • Strong Python and Django experience
  • Hands-on experience with GCP deployment and management
  • Experience with CI/CD pipeline implementation
  • Experience developing microservices
  • Experience in backend development

Gaps:
  • Resume does not explicitly state 5+ years of Python/Django experience, although the dates imply it.
  • Lack of explicit mention of testing and code review participation.
  • Resume lacks quantification of accomplishments (e.g., scale of applications, impact of automation).

Suggestions for Improvement:
  • Explicitly state the number of years of Python/Django experience in the summary or experience sections.
  • Add a bullet point detailing participation in code reviews and testing practices.
  • Quantify achievements whenever possible, using metrics to showcase impact.
  • Provide more details on the specific GCP services used and the complexity of the microservices architecture.
  • Consid

In [18]:
def save_evaluation_to_file(result_json, filename="resume_match_evaluation.txt"):
    with open(filename, "w") as file:
        file.write("Resume Match Evaluation\n")
        file.write(f"Match Score: {result_json['match_score']} / 100\n\n")

        file.write("Strengths:\n")
        for s in result_json["strengths"]:
            file.write(f"  - {s}\n")

        file.write("\nGaps:\n")
        for g in result_json["gaps"]:
            file.write(f"  - {g}\n")

        file.write("\nSuggestions for Improvement:\n")
        for i in result_json["improvement_suggestions"]:
            file.write(f"  - {i}\n")

    print(f"\nEvaluation saved to '{filename}'")

In [19]:
save_evaluation_to_file(result_json)


Evaluation saved to 'resume_match_evaluation.txt'
